<a href="https://colab.research.google.com/github/mech0s/nodehenge/blob/main/doc2rdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
revision = 10
print ("Revision", revision)

##  Install steps
Need to re-run for each fresh google colab session

In [ ]:
### commented out - rdfpandas not needed?
#  %pip install rdfpandas
#  %pip install openpyxl
%pip install rdflib
%pip install -U spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

##   Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import json

### Optional    google  colab   enhancements

In [ ]:
#optional
if 'COLAB_JUPYTER_TOKEN' in os.environ:
  from google.colab import data_table
  data_table.enable_dataframe_formatter()

## Source Data Read/Setup

### Read dodcio DevSecOpsActivitesToolsGuidebookTables.xlsx

In [ ]:
def cleanCamel(r):
  r = re.sub('\W|^(?=\d)','_', r) ## cleans to make a valid identifier
  r = re.sub(r"(_|-|\n|!)+", " ", r).title().replace(" ", "")  ## turns to camel-case
  return ''.join([r[0].lower(), r[1:]])

In [ ]:
sourceURI = "https://dodcio.defense.gov/Portals/0/Documents/Library/DevSecOpsActivitesToolsGuidebookTables.xlsx"
xls = pd.ExcelFile(sourceURI)

### Setup fixed data

In [ ]:
phaseNames = [ 'Plan',
 'Develop',
 'Build',
 'Test',
 'Release',
 'Deliver',
 'Deploy',
 'Operate',
 'Monitor',
 'Feedback']
phaseIDs = list(map(cleanCamel, phaseNames))

##  Build phaseActivityDataFrame

One named worksheet per phase: Pull these into a list of DataFrames, adjusting column names to create valid identifiers. Turn NaN entries into blank strings.

In [ ]:
phaseDataList = []
phaseOrder = 0
for phName in phaseNames:
  df = pd.read_excel(xls,phName)
  df = df.rename(columns={"Activities":"Activity",
                     "Security / Testing / CM": "SecurityTestingCM",
                     "Tool Dependencies": "ToolDependency",
                     "Tool Dependency": "ToolDependency"
                     })
  # add Phase and order columns - alternative avoid tricky MultiIndex when concatening below
  df["Phase"] = phName
  df["PhaseOrder"] = phaseOrder
  df["OrderInPhase"] = df.index
  phaseOrder+=1
  #
  phaseDataList.append(df.replace(np.nan, ""))

phaseDataList : list of dataframes :- concatenate into one dataframe

In [ ]:
phaseActivityDataFrame = pd.concat(phaseDataList)
phaseActivityDataFrame.reset_index(drop=True, inplace=True)
# MultiIndex example: phaseActivityDataFrame = pd.concat(phaseDataList,keys=phaseNames, names=["Phase","IDinPhase"])

In [ ]:
phaseActivityDataFrame["ActivityIdentifier"] = phaseActivityDataFrame["Activity"].apply( cleanCamel )

In [ ]:
phaseActivityDataFrame.to_json(orient='table')

In [ ]:
phaseActivityDataFrame.rename(columns={"ToolDependency":"ToolDependenciesText", "Inputs":"InputsText", "Outputs":"OutputsText"},inplace=True)
## prepare empty lists to accept values parsed from text fields
phaseActivityDataFrame["ToolsList"]=[[] for _ in range(len(phaseActivityDataFrame))]
phaseActivityDataFrame["InputsList"]=[[] for _ in range(len(phaseActivityDataFrame))]
phaseActivityDataFrame["OutputsList"]=[[] for _ in range(len(phaseActivityDataFrame))]

In [ ]:
# check cheet names
xls.sheet_names

In [ ]:
toolsDataFrame = pd.read_excel(xls, "Tools")
toolsDataFrame.replace(np.nan, "", inplace=True)

In [ ]:
toolsDataFrame.rename(columns={ "Inputs":"InputsText", "Outputs":"OutputsText"},inplace=True)
toolsDataFrame["InputsList"]=[[] for _ in range(len(toolsDataFrame))]
toolsDataFrame["OutputsList"]=[[] for _ in range(len(toolsDataFrame))]

### Dataframe select, filter, order examples

In [ ]:
### Dataframe slicing examples
phaseActivityDataFrame.iloc[3:39]

In [ ]:
### Dataframe ordering and filtering examples
phaseActivityDataFrame.sort_values(["PhaseOrder","OrderInPhase"])[phaseActivityDataFrame["Phase"]>="Plan"]

In [ ]:
### Dataframe selection examples
phaseActivityDataFrame[["Phase","Baseline"]]

In [ ]:
### access to lists within cells
toolsDataFrame.iloc[0].InputsList.append(123)
toolsDataFrame.iloc[0].InputsList.clear()
toolsDataFrame.iloc[0]

### Parse out text cells

In [ ]:
#helper function - check what text isn't being extracted from a string during parsing
def remaining_text(txt, strs):
    rem = txt
    for s in strs:
        rem = rem.replace(s,'')

    return rem.replace("\n","")

### Cleansing of text list cells

In [ ]:
phaseActivityDataFramePreMods = phaseActivityDataFrame.copy()
toolsDataFramePreMdods = toolsDataFrame.copy()

inputOutputReplaces ={ "Change management" : "Change-management" ,
           "IT infrastructure asset" : "IT-infrastructure-asset",
            "Artifacts (Infrastructure as Code)" : "IAC Artifacts,",
            "Infrastructure as Code" : "IAC",
            "NIST 800-53 RMF Control Implementations" :"NIST80053RMFControl-Implementations,",
            "FIPS 199 system categorization" : "FIPS-199-system-categorization,",
            "Stakeholder inputs or feedback" : "Stakeholder-inputs, Stakeholder feedback",
            "Requirements database or documents" : "Requirements database, Requirements documents",
            "Test environment applications and infrastructure" : "Test environment applications, Test environment infrastructure",
            "Developer coding and appropriate unit, integration, etc. testing input" : "Developer coding, Unit test input, Integration test input, Other test input",
            "- Review Comments" : "Review-Comments,",
            "- Source Code Weakness Findings" : "Source Code Weakness Findings,",
            "- Version-Controlled Source Code" :"Version-Controlled Source Code,",
            "- Security Findings and Warnings" : "Security Findings, Security Warnings,",
            
            "" : "",
            "" : "",
            "" : "",
            "" : "",
            "" : "",
}
for pair in inputOutputReplaces.items():
    phaseActivityDataFrame.InputsText = phaseActivityDataFrame.apply(lambda x : x.InputsText.replace(pair[0], pair[1]), axis='columns')
    phaseActivityDataFrame.OutputsText = phaseActivityDataFrame.apply(lambda x : x.OutputsText.replace(pair[0], pair[1]), axis='columns')

toolReplaces = {
            "Monitoring tool suite" : "Monitoring-tool suite",
            "Test tool suite" : "Test-tool suite",
            "Log aggregator" : "Log-aggregator",
            "Log analysis & auditing" : "Log-analysis, Log-auditing, ",
            "Logging" : "Logging-tool",
            "Logging-tool tool" : "Logging-tool",
            "" : "",
            "" : "",
}
for pair in toolReplaces.items():
    phaseActivityDataFrame.ToolDependenciesText = phaseActivityDataFrame.apply(lambda x : x.ToolDependenciesText.replace(pair[0], pair[1]), axis='columns')
    toolsDataFrame.Tool = toolsDataFrame.apply(lambda x : x.Tool.replace(pair[0], pair[1]), axis='columns')


# nlp extraction of noun phrases
phaseActivityDataFrame.InputsList = phaseActivityDataFrame.apply(lambda x :  [chunk.text for chunk in nlp(x.InputsText).noun_chunks], axis='columns')
phaseActivityDataFrame.OutputsList = phaseActivityDataFrame.apply(lambda x :  [chunk.text for chunk in nlp(x.OutputsText).noun_chunks], axis='columns')
phaseActivityDataFrame.ToolsList = phaseActivityDataFrame.apply(lambda x :  [chunk.text for chunk in nlp(x.ToolDependenciesText).noun_chunks], axis='columns')
# text left behind
phaseActivityDataFrame['InputsTextRemainder'] = phaseActivityDataFrame.apply(lambda x : remaining_text(x.InputsText, x.InputsList) , axis='columns')
phaseActivityDataFrame['OutputsTextRemainder'] = phaseActivityDataFrame.apply(lambda x : remaining_text(x.OutputsText, x.OutputsList) , axis='columns')
phaseActivityDataFrame['ToolsTextRemainder'] = phaseActivityDataFrame.apply(lambda x : remaining_text(x.ToolDependenciesText, x.ToolsList) , axis='columns')


### Cleansed - Now turn item names into identifiers - camelCase

In [ ]:
toolsDataFrame["ToolIdentifier"]=toolsDataFrame["Tool"].apply( cleanCamel )
phaseActivityDataFrame.InputsList = phaseActivityDataFrame.apply(lambda x : [ cleanCamel(y) for y in x.InputsList ], axis='columns' )
phaseActivityDataFrame.OutputsList = phaseActivityDataFrame.apply(lambda x : [ cleanCamel(y) for y in x.OutputsList ], axis='columns' )
phaseActivityDataFrame.ToolsList = phaseActivityDataFrame.apply(lambda x : [ cleanCamel(y) for y in x.ToolsList ], axis='columns' )



In [ ]:

#filtered column view to aid visual inspection post-cleansing
checkPAInputs = phaseActivityDataFrame[["Phase","Activity","InputsText","InputsList","InputsTextRemainder"]]
checkPAOutputs = phaseActivityDataFrame[["Phase","Activity","OutputsText","OutputsList","OutputsTextRemainder"]]
checkPATools = phaseActivityDataFrame[["Phase","Activity","ToolDependenciesText","ToolsList","ToolsTextRemainder"]]

## TODO : Other sheets / regions

# RDF creation

In [ ]:
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import SKOS, RDF, RDFS, XSD, NamespaceManager, DCTERMS # DC, DOAP, FOAF, VOID, XMLNS

ONT = Namespace("http://nodehenge.org/ont#")
INST = Namespace("http://nodehenge.org/inst#")
PHASE = Namespace("http://nodehenge.org/inst/phase#")
TOOL = Namespace("http://nodehenge.org/inst/tool#")
ACT = Namespace("http://nodehenge.org/inst/activity#")
ART = Namespace("http://nodehenge.org/inst/artifact#")
g = Graph() ###base="http://nodehenge.org/inst/")
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("skos", SKOS)
g.bind("xsd", XSD)
g.bind("ont", ONT)
g.bind("inst", INST)
g.bind("phase", PHASE)
g.bind("tool", TOOL)
g.bind("act", ACT)
g.bind("art", ART)



In [ ]:
# populate the top level INST namespace
g.add ((INST.dodDsopScheme, RDF.type, SKOS.ConceptScheme ))
g.add ((INST.dodDsopScheme, DCTERMS.title, Literal("DoD DevSecOps Abstract Phase Activites and Tools Scheme") ))
g.add ((INST.dodDsopScheme, DCTERMS.source, URIRef(sourceURI) ))
orderedPhaseBNodes = dict([(ph, BNode()) for ph in phaseIDs ])
g.add ((INST.phasing, RDF.type, SKOS.OrderedCollection))
g.add ((INST.phasing, SKOS.inScheme, INST.dodDsopScheme))
g.add ((INST.phasing, SKOS.memberList, orderedPhaseBNodes[phaseIDs[0]]))

In [ ]:
#populate the PHASE namespace
sideEffectTranversePhases = [ (
    g.add((
        PHASE[current], RDF.type, ONT.Phase
    )),
    g.add ((
        PHASE[current], SKOS.inScheme, INST.dodDsopScheme
    )),
    g.add((
        PHASE[current], RDF.type, SKOS.Concept
    )),
    g.add((
        PHASE[current], SKOS.prefLabel, Literal(currentName)
    )),    
    g.add((
        orderedPhaseBNodes[current], RDF.first, PHASE[current]    #list head
    )),
    g.add((
        orderedPhaseBNodes[current], RDF.rest , orderedPhaseBNodes[next] if next != None else RDF.nil
    )),
    )
    for current, currentName, next in zip( phaseIDs, phaseNames , phaseIDs[1:]+[None]) ]

#print(g.serialize(format="turtle"))

### Load

... pre-load from onto.ttl here ?
g.parse('publish/nodehenge.org/ont.ttl')

In [ ]:
toolsDataFrame.apply( lambda row : 
    (
        g.add((
            TOOL[row.ToolIdentifier], RDF.type, ONT.AbstractTool
        )),
        g.add((
            TOOL[row.ToolIdentifier], RDF.type, SKOS.Concept
        )),
        g.add ((
            TOOL[row.ToolIdentifier], SKOS.inScheme, INST.dodDsopScheme
        )),
        g.add((
            TOOL[row.ToolIdentifier], SKOS.prefLabel, Literal(row.Tool)   
        )),
        g.add((
            TOOL[row.ToolIdentifier], SKOS.definition, Literal(row.Benefits)
        )),
        g.add((
            TOOL[row.ToolIdentifier], SKOS.scopeNote, Literal(row.Features)
        )),
        g.add((
            TOOL[row.ToolIdentifier], SKOS.editorialNote, Literal("InputsText and OutputText source fields need cleansed and turned into related Atrifact entities")
        )),
    )
    , axis='columns' )
None

### TODO : Tools : InputsText and OutputText source fields need cleansed and turned into related Atrifact entities

In [ ]:
phaseActivityDataFrame.apply( lambda row : 
    (
        g.add((
            ACT[row.ActivityIdentifier], RDF.type, ONT.AbstractActivity
        )),
        g.add((
            ACT[row.ActivityIdentifier], RDF.type, SKOS.Concept
        )),
        g.add((
            ACT[row.ActivityIdentifier], SKOS.inScheme, INST.dodDsopScheme
        )),
        g.add((
            ACT[row.ActivityIdentifier], SKOS.prefLabel, Literal(row.Activity)   
        )),
        g.add((
            ACT[row.ActivityIdentifier], SKOS.definition, Literal(row.Description)
        )),
        g.add((
            ACT[row.ActivityIdentifier], SKOS.scopeNote, Literal("Baseline: " + row.Baseline )
        )),
        g.add((
            ACT[row.ActivityIdentifier], SKOS.scopeNote, Literal("Relevance: " + row.SecurityTestingCM )
        )),
        g.add((
            ACT[row.ActivityIdentifier], SKOS.scopeNote, Literal("SSDF: " + row.SSDF )
        )),
        g.add((
            ACT[row.ActivityIdentifier], SKOS.editorialNote, Literal("skos:scopeNote currently holding three fields. Could normalize")
        )),
        [ ( g.add(( ART[item], RDF.type, ONT.AbstractArtifact )),
            g.add(( ART[item], RDF.type, SKOS.Concept )),
            g.add(( ART[item], SKOS.inScheme, INST.dodDsopScheme )),
            g.add(( ACT[row.ActivityIdentifier], ONT.input, ART[item] )), 
            g.add(( ART[item], ONT.input, ACT[row.ActivityIdentifier] )),
            ) for item in row.InputsList ],
        [ ( g.add(( ART[item], RDF.type, ONT.AbstractArtifact )),
            g.add(( ART[item], RDF.type, SKOS.Concept )),
            g.add(( ART[item], SKOS.inScheme, INST.dodDsopScheme )),
            g.add(( ACT[row.ActivityIdentifier], ONT.output, ART[item] )), 
            g.add(( ART[item], ONT.output, ACT[row.ActivityIdentifier] )),
            ) for item in row.OutputsList ],
        [ ( g.add(( TOOL[item], RDF.type, TOOL.AbstractTool )),
            g.add(( TOOL[item], RDF.type, SKOS.Concept )),
            g.add(( TOOL[item], SKOS.inScheme, INST.dodDsopScheme )),
            g.add(( ACT[row.ActivityIdentifier], ONT.activityTool, TOOL[item] )), 
            g.add(( TOOL[item], ONT.activityTool, ACT[row.ActivityIdentifier] )),
            ) for item in row.ToolsList ],
        g.add((
            ACT[row.ActivityIdentifier], ONT.phaseActivity, PHASE[cleanCamel(row.Phase)]
        )),
        g.add((
            PHASE[cleanCamel(row.Phase)], ONT.phaseActivity, ACT[row.ActivityIdentifier]
        )),
        g.add(( ACT[row.ActivityIdentifier] , SKOS.editorialNote, Literal("Unused Inputs text::"+row.InputsTextRemainder))) if any(letter.isalpha() for letter in row.InputsTextRemainder) else None,
        g.add(( ACT[row.ActivityIdentifier] , SKOS.editorialNote, Literal("Unused Outputs text::"+row.OutputsTextRemainder))) if any(letter.isalpha() for letter in row.OutputsTextRemainder) else None,
        g.add(( ACT[row.ActivityIdentifier] , SKOS.editorialNote, Literal("Unused Tools text::"+row.ToolsTextRemainder))) if any(letter.isalpha() for letter in row.ToolsTextRemainder) else None,
    )
    , axis='columns' )
None

In [ ]:
g.serialize(format="turtle",destination="publish/nodehenge.org/inst.ttl")

In [ ]:
print(g.serialize(format="turtle"))

In [ ]:
# some commented-out RDF exploration tests
%%script false --no-raise-error

g.add((
    URIRef("#nick"),
    SKOS.prefLabel,
    Literal("Nick") #, datatype=XSD.string)
    )
)
g.add((
    URIRef("#bob"),
    SKOS.prefLabel,
    Literal("Bob") #, datatype=XSD.string)
    )
)

print(g.serialize(format="turtle"))

my_query = """
SELECT DISTINCT ?a ?b
WHERE {
    ?a skos:prefLabel "Nick" .
}"""

qres = g.query(my_query)
for row in qres:
    print(f"{row.a} ")

bob=URIRef("#bob")
print(g.value(bob,SKOS.prefLabel))
#```

In [ ]:
%%script false --no-raise-error

g.add( (PHASE.plan, SKOS.prefLabel, Literal("Plan")))
print(  PHASE.plan )
print( g.value( PHASE.plan, SKOS.prefLabel ) )

In [ ]:
%%script false --no-raise-error

phaseID = "pha+--=sfgsdfg   \n se13"
phaseID2 = "phase13432"
g.remove( (PHASE[cleanCamel(phaseID)], None, None) )  #dict notation as alternative to explicit value and dot notation
g.add( (PHASE[cleanCamel(phaseID)], SKOS.related, PHASE[phaseID2]))
print( g.value( PHASE[cleanCamel(phaseID)], SKOS.related ) )